In [ ]:
from sklearn.metrics import accuracy_score  # Accuracy metrics
import pickle


In [ ]:
# Load the model from the file
with open('model_name.pkl', 'rb') as f:
    model = pickle.load(f)


In [ ]:
# Classification every 3 Seconds
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import pickle
import time
import csv
import itertools
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# For webcam input:
cap = cv2.VideoCapture('Dataset/Testing/VideoUji/3.mp4')
pTime = 0
i = 0
while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
        break

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = pose.process(image)

    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(image, results.pose_landmarks)
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(image, str(int(fps)), (600, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    try:
        # Extract Pose landmarks
        data = results.pose_landmarks.landmark
        pose_row = list(np.array(
            [[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in data]).flatten())
        del pose_row[0:40]

        # Make Detections
        X = pd.DataFrame([pose_row])
        body_language_class = model.predict(X)[0]
        body_language_prob = model.predict_proba(X)[0]
        #print(body_language_class, body_language_prob)
        a = body_language_prob[np.argmax(body_language_prob)]
        prob = []
        prob.append(a)
        print(prob)
        # step = 60
        # d = []
        # for i, _ in enumerate(a[::step]):
        #     b = a[i*step] if (i+1)*step > len(a) else a[i*step:(i+1)*step]
        #     try:
        #         c = sum(b)/float(len(b))
        #         d.append(c)
        #     except:
        #         break
        # print(d)
        # print((sum(map(float,d)))/len(d))

        # # Get status box
        # cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

        # # Display Class
        # cv2.putText(image, 'CLASS'
        #             , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        # cv2.putText(image, body_language_class.split(' ')[0]
        #             , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # # Display Probability
        # cv2.putText(image, 'PROB'
        #             , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        # cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
        #             , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        # p = str(body_language_class.split(' ')[0])
        # #q = int((int(body_language_prob[np.argmax(body_language_prob)]*100)))
        # r = str(int(body_language_prob[0]*100))
        # s = str(int(body_language_prob[1]*100))
        # t = str(int(body_language_prob[2]*100))
        # if i % 60 == 1:
        #   with open('AccPred.csv', mode='a', newline='') as f:
        #     csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        #     csv_writer.writerow([p,r,s,t])

    except:
        pass

    # Displaying the image
    cv2.imshow("Image", image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [ ]:
# Normal
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import pickle, time, csv, itertools
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# For webcam input:
cap = cv2.VideoCapture('Dataset/Video/3.mp4')
width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter('Demo4.mp4', fourcc, fps, size)
pTime = 0
i = 0
while cap.isOpened():
  success, image = cap.read()
  if not success:
    print("Ignoring empty camera frame.")
    # If loading a video, use 'break' instead of 'continue'.
    break
  
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   

  results = pose.process(image)

  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
  mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

  try: 
    # Extract Pose landmarks
    data = results.pose_landmarks.landmark
    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in data]).flatten())
    del pose_row[0:40]

    # Make Detections
    X = pd.DataFrame([pose_row])
    body_language_class = model.predict(X)[0]
    body_language_prob = model.predict_proba(X)[0]
    if body_language_prob[np.argmax(body_language_prob)] >= 0.6:
      # Get status box
      cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
      
      # Display Class
      cv2.putText(image, 'CLASS'
                  , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
      cv2.putText(image, body_language_class.split(' ')[0]
                  , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
      
      # Display Probability
      cv2.putText(image, 'PROB'
                  , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
      cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                  , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)      
    else:
      pass
    # p = str(body_language_class.split(' ')[0])
    # #q = int((int(body_language_prob[np.argmax(body_language_prob)]*100)))
    # r = str(int(body_language_prob[0]*100))
    # s = str(int(body_language_prob[1]*100))
    # t = str(int(body_language_prob[2]*100))
    # with open('AccPred.csv', mode='a', newline='') as f:
    #   csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    #   csv_writer.writerow([p,r,s,t])
  except:
    # with open('AccPred.csv', mode='a', newline='') as f:
    #   csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    #   csv_writer.writerow([0,0,0,0])
    pass

  # Displaying the image
  out.write(image)
  #cv2.imshow("Image", image)
  if cv2.waitKey(1) & 0xFF == ord('q'):
          break
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# Reading File CSV
import pandas as pd
df = pd.read_csv('AccPred.csv')
a = df['picking']

In [ ]:
# Predicting/Seconds(%)
step = 30
d = []
for i, _ in enumerate(a[::step]):
    b = a[i*step] if (i+1)*step > len(a) else a[i*step:(i+1)*step]
    try:
        c = sum(b)/float(len(b))
        d.append(c)
    except:
        break
print(d)
print((sum(map(float,d)))/len(d))
    

In [ ]:
# Predicting(%)
i = 0
b = 0
a = 0
for i in range(len(df)-1):
    a += df['picking'][i]
b = (a/len(df))
print(b)

Pred/S
95.285
92.745
84.9
95.046
96.792
87.095

Pred
95.172
92.35
84.853
95.199
96.537
87.420